In [3]:
import os
import sys


In [4]:
%pwd

'/mnt/data1/Machine-Learning-End-To-End-Project/research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'/mnt/data1/Machine-Learning-End-To-End-Project'

In [ ]:
import os
import sys
from src.constants import *
from src.utils.common import read_yaml, create_directories
from urllib.request import urlretrieve
from src.loggers import logger
import zipfile
from dataclasses import dataclass
from pathlib import Path

In [13]:
@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir,
        )
        return data_ingestion_config

In [15]:
class DataIngestion:
    def __init__(self, config: DataIngestionconfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file,
            )
            logger.info(f"{filename} download with following info:\n {headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)

In [16]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e


[2025-11-24 17:38:01,908] [INFO] YAML file loaded successfully: config/config.yaml
[2025-11-24 17:38:01,919] [INFO] YAML file loaded successfully: params.yaml
[2025-11-24 17:38:01,921] [INFO] YAML file loaded successfully: schema.yaml
[2025-11-24 17:38:01,923] [INFO] Directory created or already exists: artifacts
[2025-11-24 17:38:01,925] [INFO] Directory created or already exists: artifacts/data_ingestion


[2025-11-24 17:38:02,743] [INFO] artifacts/data_ingestion/data.zip download with following info:
 Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B17D:289F7E:73C594:CE2B1B:69244AA2
Accept-Ranges: bytes
Date: Mon, 24 Nov 2025 12:08:02 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10233-MAA
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1763986082.346388,VS0,VE290
Vary: Authorization,Accept-Encoding
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: 7b3f9e4d6b6a5e2d9a490786bfb4d4fa36d2301c
Expires: Mon, 24 Nov 2025 12:13:02 GMT
Source-Age: 0


